# 初期設定

In [1]:
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests #HTTP操作用
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

/Users/yutomatsuda/Lecture/DSProg2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
wait = WebDriverWait(options,10)

# スクレイピング

In [3]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://www.google.co.jp/"
driver.get(url)
time.sleep(3)
print("サイトのタイトル：", driver.title)

サイトのタイトル： Google


In [4]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name=driver.find_element(By.XPATH,f'//*[@id="tr_{a}"]/td[4]/div/div/span/a').text
        print(house_name)
        house_link_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_link = house_link_element.get_attribute("href")
        print(house_link)

    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue

driver.quit()

ウインマリリン
https://db.netkeiba.com/horse/2017101431
ディープボンド
https://db.netkeiba.com/horse/2017102170
ホウオウエミーズ
https://db.netkeiba.com/horse/2017104691
ヒートオンビート
https://db.netkeiba.com/horse/2017104936
アイアンバローズ
https://db.netkeiba.com/horse/2017105477
タイトルホルダー
https://db.netkeiba.com/horse/2018103559
シャフリヤール
https://db.netkeiba.com/horse/2018105165
スルーセブンシーズ
https://db.netkeiba.com/horse/2018105269
プラダリア
https://db.netkeiba.com/horse/2019100109
ライラック
https://db.netkeiba.com/horse/2019103588
スターズオンアース
https://db.netkeiba.com/horse/2019104740
ドウデュース
https://db.netkeiba.com/horse/2019105283
ジャスティンパレス
https://db.netkeiba.com/horse/2019105346
ソールオリエンス
https://db.netkeiba.com/horse/2020102899
ハーパー
https://db.netkeiba.com/horse/2020103458
タスティエーラ
https://db.netkeiba.com/horse/2020103532


In [5]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = "https://db.netkeiba.com/horse/2017101431"
driver.get(url)
time.sleep(0.1)

winscore_list = []
wintime_list = []
last_list = []
weight_change_list = []

b = 1
while True:
    try:
        # 勝率の取得
        winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
        if winscore and winscore.isdigit():  # 数字のみを確認
            winscore_list.append(float(winscore))

        # 着差の取得（空白や**を回避）
        wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
        if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
            wintime_list.append(float(wintime))

        # 上りの取得（空白や**を回避）
        last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
        if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
            last_list.append(float(last))

        # 体重の増減の取得（空白を回避）
        weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
        match = re.search(r'\((\+|-)?(\d+)\)', weight)
        if match:  # マッチするものがあった場合
            weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
            weight_change_list.append(weight_change)

        b += 1

    except NoSuchElementException:
        # 要素が見つからない場合はループを終了
        break

# 数値データの平均を計算して出力
print("勝率平均:", np.mean(winscore_list) if winscore_list else "データなし")
print("着差平均:", np.mean(wintime_list) if wintime_list else "データなし")
print("上り平均:", np.mean(last_list) if last_list else "データなし")
print("馬体重増減平均:", np.mean(weight_change_list) if weight_change_list else "データなし")

driver.quit()

勝率平均: 5.476190476190476
着差平均: 1.05
上り平均: 36.472222222222214
馬体重増減平均: 1.0


In [6]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://db.netkeiba.com/horse/2017105477"
driver.get(url)
time.sleep(0.1)

father_link_list = []
f_grandfather_link_list = []
f_grandmother_link_list = []
mother_link_link = []
m_grandfather_link_list = []
m_grandmother_link_list = []

father = driver.find_element(By.XPATH, '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[1]/a') # 父
f_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[2]/a') # 父→父
f_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[2]/td/a') # 父→母
mother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[1]/a') # 母
m_grandfather = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[2]/a') # 母→父
m_grandmother = driver.find_element(By.XPATH,'//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[4]/td/a') # 母→母

house_link_father = father.get_attribute("href")
house_link_father = house_link_father.replace("ped","result")
house_link_f_grandfather = f_grandfather.get_attribute("href")
house_link_f_grandfather = house_link_f_grandfather.replace("ped","result")
house_link_f_grandmother = f_grandmother.get_attribute("href")
house_link_f_grandmother = house_link_f_grandmother.replace("ped","result")
house_link_mother = mother.get_attribute("href")
house_link_mother = house_link_mother.replace("ped","result")
house_link_m_grandfather = m_grandfather.get_attribute("href")
house_link_m_grandfather = house_link_m_grandfather.replace("ped","result")
house_link_m_grandmother = m_grandmother.get_attribute("href")
house_link_m_grandmother = house_link_m_grandmother.replace("ped","result")

father_link_list.append(house_link_father)
f_grandfather_link_list.append(house_link_f_grandfather)
f_grandmother_link_list.append(house_link_f_grandmother)
mother_link_link.append(house_link_mother)
m_grandfather_link_list.append(house_link_m_grandfather)
m_grandmother_link_list.append(house_link_m_grandmother)

In [7]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
#urlの指定
url="https://db.netkeiba.com/horse/result/2008102636/"
driver.get(url)
time.sleep(0.1)
winsocore_list_1 = []

c = 1
while True:
    try:
        winsocore_1 = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[2]/div[1]/table/tbody/tr[{c}]/td[12]').text
        winsocore_1 = float(winsocore_1)
        winsocore_list_1.append(float(winsocore_1))
        c += 1

    except NoSuchElementException:
        # 要素が見つからない場合はループを終了
        break

# 取得した数値データの平均を計算
average_value = np.mean(winsocore_list_1)
print("平均値:", average_value)

平均値: 2.2857142857142856


In [9]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import numpy as np
import time
import re

# WebDriverのインスタンスを一度だけ生成
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# URLを開く
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

# 全ての馬のデータを格納するリスト
all_data = []

# 馬のリンクを取得
houselink_list = []
for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_name = house_name_element.text
        house_link = house_name_element.get_attribute("href")
        houselink_list.append((house_name, house_link))
        print(house_name)
    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue

for house_name, house_link in houselink_list:
    # 既存のブラウザセッションを使ってリンクを開く
    driver.get(house_link)
    time.sleep(0.1)
    
    # 新しいdatum辞書を作成
    datum = {'馬名': house_name}
    winscore_list = []
    wintime_list = []
    last_list = []
    weight_change_list = []

    b = 1
    while True:
        try:
            # 勝率の取得
            winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
            if winscore and winscore.isdigit():  # 数字のみを確認
                winscore_list.append(float(winscore))

            # 着差の取得（空白や**を回避）
            wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
            if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                wintime_list.append(float(wintime))

            # 上りの取得（空白や**を回避）
            last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
            if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                last_list.append(float(last))

            # 体重の増減の取得（空白を回避）
            weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
            match = re.search(r'\((\+|-)?(\d+)\)', weight)
            if match:  # マッチするものがあった場合
                weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
                weight_change_list.append(weight_change)

            b += 1

        except NoSuchElementException:
            # 要素が見つからない場合はループを終了
            break

    # 各データの平均値を計算して辞書に追加
    datum['勝率平均'] = np.mean(winscore_list) if winscore_list else None
    datum['着差平均'] = np.mean(wintime_list) if wintime_list else None
    datum['上り平均'] = np.mean(last_list) if last_list else None
    datum['馬体重増減平均'] = np.mean(weight_change_list) if weight_change_list else None

    # 個々の馬のデータ辞書をリストに追加
    all_data.append(datum)

# 全てのデータを出力
for horse_data in all_data:
    print(horse_data)

# ドライバーを閉じる
driver.quit()

ウインマリリン
ディープボンド
ホウオウエミーズ
ヒートオンビート
アイアンバローズ
タイトルホルダー
シャフリヤール
スルーセブンシーズ
プラダリア
ライラック
スターズオンアース
ドウデュース
ジャスティンパレス
ソールオリエンス
ハーパー
タスティエーラ
{'馬名': 'ウインマリリン', '勝率平均': 5.476190476190476, '着差平均': 1.05, '上り平均': 36.472222222222214, '馬体重増減平均': 1.0}
{'馬名': 'ディープボンド', '勝率平均': 5.12, '着差平均': 0.6736842105263158, '上り平均': 35.82727272727272, '馬体重増減平均': 0.2727272727272727}
{'馬名': 'ホウオウエミーズ', '勝率平均': 5.379310344827586, '着差平均': 0.652, '上り平均': 35.455172413793115, '馬体重増減平均': -0.06896551724137931}
{'馬名': 'ヒートオンビート', '勝率平均': 3.1923076923076925, '着差平均': 0.5608695652173913, '上り平均': 34.98461538461538, '馬体重増減平均': 0.46153846153846156}
{'馬名': 'アイアンバローズ', '勝率平均': 5.555555555555555, '着差平均': 0.8958333333333334, '上り平均': 35.51111111111111, '馬体重増減平均': 1.1111111111111112}
{'馬名': 'タイトルホルダー', '勝率平均': 3.8823529411764706, '着差平均': 0.7555555555555555, '上り平均': 35.8625, '馬体重増減平均': 0.23529411764705882}
{'馬名': 'シャフリヤール', '勝率平均': 3.3846153846153846, '着差平均': 0.6222222222222222, '上り平均': 34.75555555555556, '馬体重増減平均': -1.1111111111111112}
{'馬

# 統合

In [10]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import numpy as np
import time
import re

# WebDriverのインスタンスを一度だけ生成
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# URLを開く
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

# 全ての馬のデータを格納するリスト
all_data = []

# 馬のリンクを取得
houselink_list = []
for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_name = house_name_element.text
        house_link = house_name_element.get_attribute("href")
        houselink_list.append((house_name, house_link))
    except NoSuchElementException:
        continue

# 各馬の詳細ページで情報を取得
for house_name, house_link in houselink_list:
    driver.get(house_link)
    time.sleep(0.1)

    # 新しいdatum辞書を作成
    datum = {'馬名': house_name}
    winscore_list = []
    wintime_list = []
    last_list = []
    weight_change_list = []

    # ここから情報を取得
    b = 1
    while True:
        try:
            # 勝率の取得
            winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
            if winscore and winscore.isdigit():  # 数字のみを確認
                winscore_list.append(float(winscore))

            # 着差の取得（空白や**を回避）
            wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
            if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                wintime_list.append(float(wintime))

            # 上りの取得（空白や**を回避）
            last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
            if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                last_list.append(float(last))

            # 体重の増減の取得（空白を回避）
            weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
            match = re.search(r'\((\+|-)?(\d+)\)', weight)
            if match:  # マッチするものがあった場合
                weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
                weight_change_list.append(weight_change)


            b += 1
        except NoSuchElementException:
            break

    # 各データの平均値を計算して辞書に追加
    datum['勝率平均'] = np.mean(winscore_list) if winscore_list else None
    datum['着差平均'] = np.mean(wintime_list) if wintime_list else None
    datum['上り平均'] = np.mean(last_list) if last_list else None
    datum['馬体重増減平均'] = np.mean(weight_change_list) if weight_change_list else None

    # 血統情報のリンクを取得して、その勝率を計算
    bloodline_links = []
    try:
        bloodline_elements = {
            '父': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[1]/a',
            '父の父': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[1]/td[2]/a',
            '父の母': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[2]/td/a',
            '母': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[1]/a',
            '母の父': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[3]/td[2]/a',
            '母の母': '//*[@id="db_main_box"]/div[2]/div/div[2]/div/dl/dd/table/tbody/tr[4]/td/a'
        }
        for key, xpath in bloodline_elements.items():
            element = driver.find_element(By.XPATH, xpath)
            link = element.get_attribute("href").replace("ped", "result")
            bloodline_links.append((key, link))
    except NoSuchElementException:
        continue

    # 血統リンクから勝率を取得
    for key, link in bloodline_links:
        driver.get(link)
        time.sleep(0.1)
        winscore_list = []
        c = 1
        while True:
            try:
                winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[2]/div[1]/table/tbody/tr[{c}]/td[12]').text
                if winscore and winscore.isdigit():
                    winscore_list.append(float(winscore))
                c += 1
            except NoSuchElementException:
                break

        datum[f'{key}の勝率平均'] = np.mean(winscore_list) if winscore_list else None

    # 全体のデータリストに個々の馬のデータを追加
    all_data.append(datum)

# 全てのデータを出力
for horse_data in all_data:
    print(horse_data)

# ドライバーを閉じる
driver.quit()


{'馬名': 'ウインマリリン', '勝率平均': 5.476190476190476, '着差平均': 1.05, '上り平均': 36.472222222222214, '馬体重増減平均': 1.0, '父の勝率平均': 4.3478260869565215, '父の父の勝率平均': 2.8666666666666667, '父の母の勝率平均': 7.0, '母の勝率平均': 5.777777777777778, '母の父の勝率平均': None, '母の母の勝率平均': None}
{'馬名': 'ディープボンド', '勝率平均': 5.12, '着差平均': 0.6736842105263158, '上り平均': 35.82727272727272, '馬体重増減平均': 0.2727272727272727, '父の勝率平均': 2.5, '父の父の勝率平均': 1.0769230769230769, '父の母の勝率平均': None, '母の勝率平均': 5.681818181818182, '母の父の勝率平均': 5.037037037037037, '母の母の勝率平均': 6.0}
{'馬名': 'ホウオウエミーズ', '勝率平均': 5.379310344827586, '着差平均': 0.652, '上り平均': 35.455172413793115, '馬体重増減平均': -0.06896551724137931, '父の勝率平均': 1.368421052631579, '父の父の勝率平均': 1.25, '父の母の勝率平均': 6.0, '母の勝率平均': 4.111111111111111, '母の父の勝率平均': 1.0, '母の母の勝率平均': 13.0}
{'馬名': 'ヒートオンビート', '勝率平均': 3.1923076923076925, '着差平均': 0.5608695652173913, '上り平均': 34.98461538461538, '馬体重増減平均': 0.46153846153846156, '父の勝率平均': 1.25, '父の父の勝率平均': None, '父の母の勝率平均': None, '母の勝率平均': 5.954545454545454, '母の父の勝率平均': 1.07692307692307

In [11]:
df = pd.DataFrame(all_data)
df

,馬名,勝率平均,着差平均,上り平均,馬体重増減平均,父の勝率平均,父の父の勝率平均,父の母の勝率平均,母の勝率平均,母の父の勝率平均,母の母の勝率平均
0,ウインマリリン,5.476190,1.050000,36.472222,1.000000,4.347826,2.866667,7.000000,5.777778,NaN,NaN
1,ディープボンド,5.120000,0.673684,35.827273,0.272727,2.500000,1.076923,NaN,5.681818,5.037037,6.000000
2,ホウオウエミーズ,5.379310,0.652000,35.455172,-0.068966,1.368421,1.250000,6.000000,4.111111,1.000000,13.000000
3,ヒートオンビート,3.192308,0.560870,34.984615,0.461538,1.250000,NaN,NaN,5.954545,1.076923,NaN
4,アイアンバローズ,5.555556,0.895833,35.511111,1.111111,2.285714,3.937500,8.391304,NaN,NaN,NaN
5,タイトルホルダー,3.882353,0.755556,35.862500,0.235294,1.444444,1.250000,4.095238,4.681818,3.000000,NaN
6,シャフリヤール,3.384615,0.622222,34.755556,-1.111111,1.076923,NaN,NaN,NaN,NaN,NaN
7,スルーセブンシーズ,3.846154,0.462500,35.058333,1.666667,4.645161,3.937500,8.391304,6.259259,1.900000,NaN
8,プラダリア,3.692308,0.481818,34.969231,0.923077,1.076923,NaN,NaN,8.461538,1.900000,9.142857
9,ライラック,6.846154,0.818182,34.792308,0.615385,2.285714,3.937500,8.391304,8.041667,1.250000,6.875000


In [12]:
import sqlite3

In [13]:
!pwd

/Users/yutomatsuda/Lecture/DSProg2


In [14]:
# DBファイルを保存するためのファイルパス

# Google Colab
path = '/Users/yutomatsuda/Lecture/DSProg2/'

# DBファイル名
db_name = 'Keiba.sqlite'

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)

# 2．SQLを実行するためのオブジェクトを取得
cur = con.cursor()

# 3．実行したいSQLを用意する
# テーブルを作成するSQL
# CREATE TABLE テーブル名（カラム名 型，...）;
sql_create_table_DSprogHW = 'CREATE TABLE Keiba(house_name text, win_score real, wintime real, last real, weight real, F real, FF real, FM real, M real, MF real, MM real);'

# 4．SQLを実行する
cur.execute(sql_create_table_DSprogHW)

In [15]:
# データを挿入するSQLコマンド
sql_insert_data = 'INSERT INTO Keiba (house_name, win_score, wintime, last, weight, F, FF, FM, M, MF, MM) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);'

# データリストからデータをテーブルに挿入
for data in all_data:
    try:
        # 辞書からデータを取り出し、データベースに挿入
        cur.execute(sql_insert_data, (
            data['馬名'],
            data['勝率平均'],
            data['着差平均'],
            data['上り平均'],
            data['馬体重増減平均'],
            data.get('父の勝率平均', 'N/A'),
            data.get('父の父の勝率平均', 'N/A'),
            data.get('父の母の勝率平均', 'N/A'),
            data.get('母の勝率平均', 'N/A'),
            data.get('母の父の勝率平均', 'N/A'),
            data.get('母の母の勝率平均', 'N/A')  
        ))
    except KeyError as e:
        print(f"データにキーが欠けています: {e}")

# 変更をコミット
con.commit()

In [16]:
# データを選択するSQLコマンド
sql_select_data = 'SELECT * FROM Keiba;'
cur.execute(sql_select_data)

# すべての結果を取得
all_results = cur.fetchall()

# 結果を表示
for result in all_results:
    print(result)

# データベース接続を閉じる
con.close()

('ウインマリリン', 5.476190476190476, 1.05, 36.472222222222214, 1.0, 4.3478260869565215, 2.8666666666666667, 7.0, 5.777777777777778, None, None)
('ディープボンド', 5.12, 0.6736842105263158, 35.82727272727272, 0.2727272727272727, 2.5, 1.0769230769230769, None, 5.681818181818182, 5.037037037037037, 6.0)
('ホウオウエミーズ', 5.379310344827586, 0.652, 35.455172413793115, -0.06896551724137931, 1.368421052631579, 1.25, 6.0, 4.111111111111111, 1.0, 13.0)
('ヒートオンビート', 3.1923076923076925, 0.5608695652173913, 34.98461538461538, 0.46153846153846156, 1.25, None, None, 5.954545454545454, 1.0769230769230769, None)
('アイアンバローズ', 5.555555555555555, 0.8958333333333334, 35.51111111111111, 1.1111111111111112, 2.2857142857142856, 3.9375, 8.391304347826088, None, None, None)
('タイトルホルダー', 3.8823529411764706, 0.7555555555555555, 35.8625, 0.23529411764705882, 1.4444444444444444, 1.25, 4.095238095238095, 4.681818181818182, 3.0, None)
('シャフリヤール', 3.3846153846153846, 0.6222222222222222, 34.75555555555556, -1.1111111111111112, 1.076923

In [17]:
# CSVファイル名
csv_file_name = 'horse_data.csv'

# CSVファイルを書き出し
df.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

print(f'{csv_file_name} にデータが書き出されました。')

horse_data.csv にデータが書き出されました。
